In [ ]:
#python3 t2_isupdata_s1.py

In [ ]:
#takes cancerous area from the image
#for radboud takes all cancer area, no diffirentiation by gs class

In [1]:
import os, gc, subprocess, time, sys, shutil

import scipy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from tqdm import tqdm
#from tqdm.notebook import tqdm
import skimage.io
from skimage.transform import resize, rescale
from math import sqrt
import statistics

sys.path.append("..")

In [2]:
import PIL.Image
#PIL.Image.MAX_IMAGE_PIXELS = 400000000
PIL.Image.MAX_IMAGE_PIXELS = None

In [3]:
#%matplotlib inline

In [4]:
from panda_bvv_config import *

## 1. Inputs ##

In [5]:
Zoom = 1
input_folder = train_size1
input_mask_folder = mask_size1
new_proc_folder = cancer_s1
kw_for_tile_func = {'gauss_blur':20}

In [7]:
# wrapping inputs:
if not os.path.exists(new_proc_folder):
    print("[INFO] 'creating {}' directory".format(new_proc_folder))
    os.makedirs(new_proc_folder)


## 2. Standard functions ##

In [ ]:
#this is a traditional write down function:
def datanew_wr(output_data_path_, img_num_, data_new_):
    cv2.imwrite(os.path.join(output_data_path_, img_num_ + '.png'), data_new_)


In [9]:
#wrapper for image processing function
def process_all_images(proc_func):
    
    def wrapper(zoom = Zoom,
                input_biopsy_folder = input_folder,
                output_data_path = new_proc_folder,
                df_name = train_labels,
                tif_file = False,
                with_mask = True,
                input_mask_folder = input_mask_folder,
                **kw):
        print(input_biopsy_folder)
        print(output_data_path)
        with tqdm(total=df_name.shape[0]) as pbar:
            for  i, row in enumerate(tqdm(df_name.iterrows())):
                img_num = row[1]['image_id']
                provider = row[1]['data_provider']                         
                mask = None
                if tif_file:
                    try:
                        biopsy = skimage.io.MultiImage(os.path.join(input_biopsy_folder,
                                                                    img_num + '.tiff'))[zoom]               
                    except:
                        print('Failed to read tiff:', img_num)                        
                else:
                    try:
                        biopsy = skimage.io.imread(os.path.join(input_biopsy_folder, img_num + '.png'))
                        shape = biopsy.shape
                        if shape[0]<shape[1]: 
                            biopsy = np.rot90(biopsy)
                            shape = biopsy.shape 
                    except:
                        print(f'can not proceed with {img_num}')   
                if with_mask:
                    try:
                        mask = skimage.io.imread(os.path.join(input_mask_folder, img_num + '_mask.png'))
                        shape = mask.shape
                        if shape[0]<shape[1]: 
                            mask = np.rot90(mask)
                            shape = mask.shape 
                    except:
                        print('Failed to process mask:', img_num) 
                try: 
                    data_new = proc_func(biopsy, 
                                         mask, 
                                         provider,
                                         1,
                                         **kw)
                    datanew_wr(output_data_path_= output_data_path,
                                img_num_ = img_num, 
                                data_new_ = data_new)                    
                                      
                except Exception as ee:
                    print('Processing mistake:\n', ee, '\n', img_num)            
                try:
                    del biopsy, mask, data_new, mask_new
                except:
                    pass
                pbar.update(1)
            gc.collect()
        return
    
    return wrapper

In [10]:
def change_karolinska_mask_for_isup_grade_direct(mask_data, chan):
    """
    This function returns pre-processed radbound mask for image multiplication
    """

    try:
        new_mask = np.zeros(mask_data.shape)#.astype('uint8')
        cancerous_tissue = False
        try:             
            mask_area = np.where(mask_data[:,:,chan] == 2)
            new_mask[mask_area[0], mask_area[1], :] = [1, 1, 1] 
            del mask_area, mask_data
            if new_mask.any() >0: 
                cancerous_tissue = True    
        except Exception as ee:
            print("Mask {%s} processing mistake: " %mask_image, ee)
    except Exception as ee:
        print("Mask {%s} read mistake: "  %mask_image, str(ee))

    gc.collect()                   
    
    return cancerous_tissue, new_mask

In [ ]:
def change_radboud_mask_for_isup_grade_direct(mask_data, chan,
                                      gauss_bl):
    """
    This function returns pre-processed radbound mask for image multiplication
    """

    try:
        new_mask = np.zeros(mask_data.shape)#.astype('uint8')
        cancerous_tissue = False
        try: 
            for area_value in range(3,5):
                mask_area = np.where(mask_data[:,:,chan] == area_value)
                new_mask[mask_area[0], mask_area[1], :] = [1, 1, 1]
                del mask_area
            if new_mask.any() >0: 
                cancerous_tissue = True
            del mask_data,    
        except Exception as ee:
            print("Mask {%s} processing mistake: " %mask_image, ee)
    except Exception as ee:
        print("Mask {%s} read mistake: "  %mask_image, str(ee))
        
    #new_mask[:,:,:] = cv2.blur(new_mask[:,:, :],(gauss_bl,gauss_bl))
    new_mask[:,:,:] = cv2.bilateralFilter(np.float32(new_mask[:,:,:]),gauss_bl,25,25)
    #medianBlur(new_mask[:,:, 2],5)
    #new_mask[:,:,:] = cv2.GaussianBlur(new_mask[:,:,:],(gauss_bl,gauss_bl), 0)
    #new_mask[:,:,:] =  np.rint(new_mask[:,:,:]) #.astype(int) 
    new_mask[:,:,:] =  np.ceil(new_mask[:,:,:]) #.astype(int) 
    #gc.collect()                   
    
    return cancerous_tissue, new_mask

In [13]:
def get_cancer_area_from_mask(cancer_mask, biopsy):
    """
    Returns multiplication of mask and image
    """
    try:
        only_cancer = np.multiply(cancer_mask,biopsy).astype(int)
    except Exception as ee:
        print(str(ee))
    
    return only_cancer  

## 3. Processing functions ##

In [1]:
@process_all_images
def get_cancer_area_for_isup_grade(biopsy, 
                                   mask, 
                                   provider, 
                                   gleason_score, 
                                   **kw):
    """
    Extracts cancer area only (if cancerous), from image and mask; or unchanged (if non-cancerous).
    Do NOT split cancer areas on Gleason.
    This function applicable for ISUP-grade based training
    Do NOT tile image
    """
    
    gauss_bl = kw.get('gauss_bl') if kw.get('gauss_bl') else 20    
    channel = 0 if len(np.unique(mask[:,:,0])) >1 else 2    
              
    if provider == 'karolinska':
        cancerous, new_mask  = change_karolinska_mask_for_isup_grade_direct(mask, chan = channel)
    elif provider == 'radboud':
        cancerous, new_mask  = change_radboud_mask_for_isup_grade_direct(mask, chan = channel,
                                      gauss_bl = gauss_bl)
    if cancerous:
        temp_im = get_cancer_area_from_mask(new_mask, biopsy)
        data_new = cv2.bitwise_not(temp_im.astype('uint8') * 255)
        #inv = cv2.bitwise_not(temp_im )
        #inv = biopsy
    else:
        data_new = biopsy

    return data_new
    

NameError: name 'process_all_images' is not defined

## 4. Process input files for training purpose ##

In [14]:
get_cancer_area_for_isup_grade(**kw_for_tile_func)

In [1]:
import os
module_name = 't2_isupdata_s1'

os.system('jupyter nbconvert --to python ' + module_name + '.ipynb')
with open(module_name + '.py', 'r') as f:
    lines = f.readlines()
with open(module_name + '.py', 'w') as f:
    for line in lines:
        if 'nbconvert --to python' in line:
            break
        else:
            f.write(line)